# Part 0:
## import everything
Run the cell below

In [ ]:
import os
import glob, types
import numpy as np
from platform import system as OS
import pandas as pd
import scipy.stats
import math
import datetime
from copy import deepcopy
import matplotlib.cm as cm
import warnings
warnings.filterwarnings("ignore")
import sys
import pickle
import string
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL
from scipy import stats
from scipy.ndimage.filters import gaussian_filter as smooth
import matplotlib.animation as animation
import matplotlib.backends.backend_pdf
import mpl_toolkits.axes_grid1.inset_locator as inset
from matplotlib.ticker import FormatStrFormatter, MaxNLocator, ScalarFormatter, FuncFormatter, PercentFormatter
from matplotlib.patches import ConnectionPatch, FancyArrowPatch, Arc
from set_rc_params import set_rc_params
import ROOT


if "__file__" not in dir():
    %matplotlib inline
    %config InlineBackend.close_figures = False

    root=ROOT.root
    
    ThisNoteBookPath=os.path.dirname(os.path.realpath("__file__"))
    CommonNoteBookesPath=os.path.join(os.path.split(ThisNoteBookPath)[0],"load_preprocess_rat")
    CWD=os.getcwd()
    os.chdir(CommonNoteBookesPath)
    %run UtilityTools.ipynb
    %run Animal_Tags.ipynb
    %run loadRat_documentation.ipynb
    %run Lesion_Size.ipynb
    %run plotRat_documentation_1_GeneralBehavior.ipynb
    %run plotRat_documentation_3_KinematicsInvestigation.ipynb
    %run RunBatchRat_3_CompareGroups.ipynb
    %run BatchRatBehavior.ipynb
    currentNbPath=os.path.join(os.path.split(ThisNoteBookPath)[0],'LesionPaper','Lick.ipynb')
    %run $currentNbPath

    os.chdir(CWD)

    logging.getLogger().setLevel(logging.ERROR)
    
    param={
        "goalTime":7,#needed for pavel data only
        "treadmillRange":[0,90],#pavel error conversion "treadmillRange":[0,80]
        "maxTrialDuration":15,
        "interTrialDuration":10,#None pavel
        "endTrial_frontPos":30,
        "endTrial_backPos":55, 
        "endTrial_minTimeSec":4,
        "cameraSamplingRate":25, #needed for new setup    

        "sigmaSmoothPosition":0.1,#0.33, 0.18 pavel
        "sigmaSmoothSpeed":0.3,#0.3, 0.5 pavel
        "nbJumpMax":100,#200 pavel
        "binSize":0.25,
        #parameters used to preprocess (will override the default parameters)
    }
    Y1,Y2=param['treadmillRange']

    print('os:',OS(),'\nroot:',root,'\nImport successful!')

---
---


# part 1:

# DEFINITIONS

### If you don't know what to do, move to part 2

In [ ]:
def add_panel_caption(axes: tuple, offsetX: tuple, offsetY: tuple, **kwargs):
    """
    This function adds letter captions (a,b,c,d) to Axes in axes
    at top left, with the specified offset, in RELATIVE figure coordinates
    """
    assert len(axes)==len(offsetX)==len(offsetY), 'Bad input!'
    
    fig=axes[0].get_figure()
    fbox=fig.bbox
    for ax,dx,dy,s in zip(axes,offsetX,offsetY,string.ascii_uppercase):
        axbox=ax.get_window_extent()
    
        ax.text(x=(axbox.x0/fbox.xmax)-abs(dx), y=(axbox.y1/fbox.ymax)+abs(dy),
                s=s,fontweight='extra bold', fontsize=10, ha='left', va='center',
               transform=fig.transFigure,**kwargs)

---

String Format for Scientific Notation

In [ ]:
def SciNote(string):
    """
    Format numbers with Real scientific notation
    Ex: 'p-val={}'.format(SciNote(p))
    """
    f = ScalarFormatter(useOffset=False, useMathText=True)
    g = lambda x,pos : "${}$".format(f._formatSciNotation('%1.2e' % x))
    fmt = FuncFormatter(g)
    return fmt(string)

---

plotting group double errorbars for speed 

In [ ]:
def plot_event_1on1(root, ax, Profiles, colorCode, goodAnimals=None, TaskParamToPlot='% good trials',
                   x_pos=None, nPre=slice(-3,None), nPost=slice(0,3),nFin=slice(3,6),
                    seed=1, animal_plot=True):
    if goodAnimals is None:
        goodAnimals,sessionDict=event_detect(root, Profiles[0], Profiles[1])
    else:
        animalList,sessionDict=event_detect(root, Profiles[0], Profiles[1])
        sessionDict={animal:sessionDict[animal] for animal in goodAnimals}
    
    if x_pos is None:
        diff=.35
        x_c=ax.get_xlim()[1]
        x_pos=(x_c-diff,x_c,x_c+diff)
    diff=x_pos[1]-x_pos[0]
    
    #getting the data
    
    
    data=np.empty((len(goodAnimals),3))
    for i,animal in enumerate(goodAnimals):
        preSession,postSession=sessionDict[animal][0], sessionDict[animal][1]
        out=animal_learning_stats(root, animal, PerfParam=TaskParamToPlot,
                                  goodSessions=[*preSession,*postSession],redo=False)
        
        data[i,0]=np.nanmedian(out[:len(preSession)][nPre])
        data[i,1]=np.nanmedian(out[len(preSession):][nPost])
        data[i,2]=np.nanmedian(out[len(preSession):][nFin])
        
            
    
    y=np.nanpercentile(data,50,axis=0)
    yerr=np.nanpercentile(data,(25,75),axis=0)
    
    #plotting the errorbar
    ax.errorbar(x_pos, y, abs(yerr-y),fmt='-o', zorder=5, ms=3, elinewidth=1, color='k')
    
    #plotting individual animals
    if animal_plot:
        np.random.seed(seed=seed)
        _coeff=5000
        for i in range(data.shape[0]):
            if np.any(np.isnan(data[i,:])):
                continue
            jitPre =np.random.uniform(low=x_pos[0]-diff/_coeff, high=x_pos[0]+diff/_coeff, size=1)
            jitPost=np.random.uniform(low=x_pos[1]-diff/_coeff, high=x_pos[1]+diff/_coeff, size=1)
            jitFin =np.random.uniform(low=x_pos[2]-diff/_coeff, high=x_pos[2]+diff/_coeff, size=1)
            
#             ax.scatter([jitPre,jitFin],data[i,[1,-1]], s=2, c=c, marker='o', edgecolors='none', alpha=.8)
            ax.plot([jitPre,jitPost,jitFin],data[i,:], c='gray', lw=.2, alpha=.4)
#             ax.plot([jitPre,jitFin],data[i,[1,-1]], c=c, lw=.4, alpha=.6)

    ax.set_xlim([x_pos[0]-diff/2,x_pos[-1]+diff/2])
    ax.set_xticks(x_pos)
    ax.set_xticklabels(['Before','Acute','Stable'])
    ax.xaxis.set_tick_params(rotation=45)
    ax.spines['bottom'].set_bounds(x_pos[0],x_pos[-1])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_ylabel(TaskParamToPlot)

    return data,goodAnimals

In [ ]:
if "__file__" not in dir():

    profile1={'Type':'Good',
             'rewardType':'Progressive',
             'Speed':['8','6'],
             'Tag':['Reverse','Reverse-AfterBreak']
             }
    profile2={'Type':'Good',
             'rewardType':'Progressive',
             'Speed':['8','6'],
             'Tag':['Reverse-Late_DLS']
             }

    #number of sessions to plot
    badAnimals=['RatBAD']
    TaskParamToPlot='median entrance time (sec)'
    wspace=0.05
    
    
    Profiles=(profile1,profile2,)
    plt.close('all')
    fig=plt.figure(figsize=(5,4))
    ax=fig.add_subplot(111)

    ax.set_xticks([])
    a=plot_event_1on1(root, ax, Profiles, colorCode={'':'xkcd:aqua','DLS':'xkcd:aqua'}, badAnimals=None, TaskParamToPlot=TaskParamToPlot,
                       x_pos=None, animal_plot=True)

---

AntiSequential trials: Starting from Back for the Reverse Treadmill

In [ ]:
def _anti_sequential_trials(data, xFront=60):
    data.position_correction()
    detect_trial_end(data,trials=data.position.keys())
    
    cs=int(data.cameraSamplingRate)
    trialStart=int(cs*data.cameraToTreadmillDelay)

    seqTrial=[]
    for trial,pos in data.position.items():
        #starting from the front
        if pos[trialStart]<xFront and pos[0]<xFront:
            continue

        #not completing the trial
        if np.isnan(data.entranceTime[trial]) or data.entranceTime[trial]==data.maxTrialDuration[trial]:
            continue

        #The trial IS indeed sequential
        seqTrial.append(trial)

    return len(seqTrial)/data.nTrial

In [ ]:
if "__file__" not in dir():
    session='Rat249_2018_08_27_12_42'
    data=Data(root,session[:6],session,redoPreprocess=False)
    lickTrdStop=plot_lick_raster(data,plot_inset=False)
    plt.close()
    posLicks=[np.array(lick)[np.array(lick)>0] for i,lick in enumerate(lickTrdStop)
              if len(lick)>0 and i in data.goodTrials]
    firstLick=np.array([i[2] for i in posLicks if len(i)>2])
#     firstLick=firstLick[firstLick> 0.2]
    print(np.median(firstLick))

---

plot the underlined text

In [ ]:
def add_underlined_text_centered_at(fig,p1,p2, s):
    
    
    fig.add_artist(mpl.lines.Line2D([p1[0],p2[0]] , [p1[1],p2[1]],
                                    lw=1,color='k'))
    
    fig.add_artist(mpl.text.Text(x=np.mean([p1[0],p2[0]]), y=np.mean([p1[1],p2[1]]),
                                        text=s, c='k',
                                        ha='center',va='bottom',fontsize='x-small'))

In [ ]:
if "__file__" not in dir():

    plt.close('all')
    fig=plt.figure(figsize=(5,5))
    ax=fig.add_subplot(111)

    
    add_underlined_text_centered_at(fig,[.2,.5],[.7,.5],'test')
    
    
    plt.show()

------



------

# part 2:

# GENERATING THE FIGURE

Definition of Parameters

In [ ]:
if "__file__" not in dir():
    # GENERAL PARAMS
    
    CtrlColor='gray'
    DLSColor='xkcd:red'
    DMSColor='xkcd:blue'
    DSColor='xkcd:magenta'
    
    ColorCode={'DS':DSColor,
               'DMS':DMSColor,
               'DLS':DLSColor,
               'Control':CtrlColor
              }
    
    BadLateRats=('Rat223','Rat231')
    colorSig='goldenrod'


        
    
    #===============================================
    
    # GRID 3 PARAMS

    session3Before='Rat222_2018_04_10_11_07'
    session3After ='Rat222_2018_04_24_11_52'
    session3Final ='Rat222_2018_05_04_10_59'
    

    
    
    #===============================================
    
    # GRID 4 PARAMs
    
    profilePre4={'Type':'Good',
                  'rewardType':'Progressive',
                  'initialSpeed':['10','0'],
                  'Speed':'10',
                  'Tag':['Control', 'Control-AfterBreak', 'Control-Late-NoTimeout-BackToTimeout', 'Control-NoTimeout-Control',
                         'Control-Sharp','IncReward-Late-Sharp','Control-Sharp-AfterBreak','ImmobileTreadmill-Control']
              }
    
    profilePost4={'Type': 'Good',
                 'rewardType': 'Progressive',
                 'option': ['not used', 'AsymmetricLesion'],
                 'initialSpeed': ['0', '10'],
                 'Speed': '10',
                 'Tag': ['Late-Lesion_DMS',
                          'Late-Lesion_DLS',
                          'Late-Lesion_DS',
                          'Late-Lesion_DS-Sharp',
                          'Late-Lesion_DMS-Sharp']
                }

    goodAnimals4=['Rat221','Rat249','Rat250','Rat263',  #DLS
                  'Rat217','Rat219','Rat224','Rat226',  #DMS
                  'Rat215','Rat218','Rat220','Rat222']  #DS
    
    TaskParamToPlot4='Lick Onset Delay'
    color4='xkcd:crimson'
    
    Profiles4=(profilePre4,profilePost4,)
    preSlice4 =slice(-2,None)
    postSlice4=slice(0,2)
    finSlice4 =slice(8,10)
    

    #===============================================
    
    # GRID 5 PARAMs
    
#     TaskParamToPlot5 = _anti_sequential_trials.__name__
    

Plotting the figure

In [ ]:
if "__file__" not in dir():
    plt.close('all')
    set_rc_params()
    figsize=(6,2)
    fig=plt.figure(figsize=figsize,dpi=600)
    
        
    
    

    ##########################################
    # 3: plot single session trajectories
    gs3= fig.add_gridspec(nrows=1, ncols=3, wspace=.1, left=0.04, bottom=0.1, right=.49, top=.4)
    ax3L= fig.add_subplot(gs3[0])
    ax3M= fig.add_subplot(gs3[1])
    ax3R= fig.add_subplot(gs3[2])
    
#     plot_trajectories_and_distributions(root, ax3L, session3Before)
#     plot_trajectories_and_distributions(root, ax3M, session3After)
#     plot_trajectories_and_distributions(root, ax3R, session3Final)

    ax3L.set_title('(#$-1$)',fontsize='xx-small')
    ax3M.set_title('(#$+1$)',fontsize='xx-small')
    ax3R.set_title('(#$+9$)',fontsize='xx-small')
    
#     ax3L.tick_params(bottom=True, left=True, labelbottom=True, labelleft=True)
#     ax3L.set_xticks([0,1,2,3,4,5,6,7,8,9,10])
#     ax3L.set_xticklabels([0,'','','','','','',7,'','',10])
#     ax3L.set_yticks(range(0,91,10))
#     ax3L.set_yticklabels([0,'','','','','','','','',90])
#     ax3L.spines['bottom'].set_bounds(0,10)
#     ax3L.set_xlabel('Trial time (s)',labelpad=0)
#     ax3L.set_ylabel('Position (cm)',labelpad=0)
#     ax3L.spines['left'].set_bounds(0,90)   
#     ax3L.spines['bottom'].set_visible(True)
#     ax3L.spines['left'].set_visible(True)




    ##########################################
    # 4: reverse lesion event plot ET
    gs4= fig.add_gridspec(nrows=1, ncols=1, left=.59, bottom=0.1, right=0.74, top=.5)
    ax4= fig.add_subplot(gs4[0])
    
    data4,animalList4=plot_event_1on1(root, ax4, Profiles4, colorCode={'':color4,'DLS':color4}, goodAnimals=goodAnimals4,
                                      x_pos=(1,2,3),TaskParamToPlot=TaskParamToPlot4, nPre=preSlice4, nPost=postSlice4,nFin=finSlice4)

    
    Eline=ax4.plot((1,2,3),data4[animalList4.index(session3Before[:6]),:],'v',
                   ms=2,mfc='white',mec='k',markeredgewidth=.5,zorder=50)
    
    
    ax4.legend([matplotlib.lines.Line2D([0,1], [0,1],color='k'),*Eline],
               [f'Group\n($n={len(animalList4)}$)','Example'],
               loc='lower right', fontsize=5,frameon=False,markerscale=1.5)


#     ax4.axhline(y=7, ls='--',c='xkcd:darkgreen',lw=.7, zorder=-5)
#     ax4.set_ylim([1,9])
#     ax4.spines['left'].set_bounds(1,9)
#     ax4.set_yticks(np.arange(1,10))
#     ax4.set_yticklabels([1,'','',4,'','',7,'',9])
#     ax4.set_ylabel('Entrance time (s)')

    
#     #STATS
    p4,s4=bootstrapTest(data4[:,-1]-data4[:,0])
    permtest.plotSigPair(ax4,y=8.8,x=(1,3), s=s4, color=colorSig,lw=.8)

    
    
    
    
    ##########################################
    # 5: reverse lesion event plot ANTI SEQ
    gs5= fig.add_gridspec(nrows=1, ncols=1, left=.81, bottom=0.1, right=.96, top=.5)
    ax5= fig.add_subplot(gs5[0])
    
#     data5,animalList5=plot_event_1on1(root, ax5, Profiles4, colorCode={'':color4,'DLS':color4}, goodAnimals=goodAnimals4, x_pos=(1,2,3,4),
#                          TaskParamToPlot=TaskParamToPlot5, nInit=initSlice4, nPre=preSlice4, nPost=postSlice4,nFin=finSlice4)

#     ax5.plot((1,2,3,4),data5[animalList5.index(session3Naive[:6]),:],'v',
#              ms=2,mfc='white',mec='k',markeredgewidth=.5,zorder=50)

    
#     ax5.set_ylim([0.2,1])
#     ax5.spines['left'].set_bounds(.2,1)
#     ax5.yaxis.set_major_formatter(mpl.ticker.PercentFormatter(xmax=1,decimals=None,symbol=None))
#     ax5.set_yticks(np.arange(.2,1.01,.2))
#     ax5.set_ylabel('%Anti routine',labelpad=0)


#     #STATS
#     p5,s5=bootstrapTest(data5[:,-1]-data5[:,1])
#     permtest.plotSigPair(ax5,y=.98,x=(2,4), s=s5, color=colorSig,lw=.8)

    


    
#     fig.align_ylabels([axes1[0],axes2[0],axes3[0],axes4[0]])
    #############################################
    #%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
#     AXES=(ax1,ax3_,ax4,ax5)
#     OFFX=np.array([.07]*len(AXES))
#     OFFY=np.array([.03]*len(AXES))
#     OFFX[0]=.02
#     OFFY[0]=.05
#     OFFY[1]=.1
    
#     add_panel_caption(axes=AXES, offsetX=OFFX, offsetY=OFFY)
    
    fig.savefig(os.path.join(os.path.dirname(os.getcwd()),'LesionPaper','Figures','Lick.pdf'),
                format='pdf', bbox_inches='tight')
    
    plt.show()
    plt.close('all')
    matplotlib.rcdefaults()